<a href="https://colab.research.google.com/github/RaghadQadah/Automatic-Spam-Review-Detection/blob/main/SpamReviewDetectionWithExport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ENCS434 Artificial Intelligence Project 2**

### **Raghad Qadah 1170142**

### **Lina Bani Oudeh 1172351**

### **Shaima Zaid 1170540**


ENCS434 Artificial Intelligence 
Programming Project 2 
Instructor: Aziz Qaroush
by:
Raghad qadah 1170142
lina bani Oudeh 1172351
shaima zaid 1170540 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## importing Library 

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score,precision_score,recall_score,f1_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# **functions to extract features from text**

In [5]:
def clean(df, text, name):
    df[name] = df[text].str.lower()
    df[name] = df[name].str.replace('http\S+|www.\S+', '', case=False)
    df[name] = df[name].str.replace('[^\w\s]', '')
    df[name] = df[name].str.replace('\d+', '')
    df[name] = df[name].apply(lambda x: ' '.join([word for word in  str(x).split() if word not in (stop)]))
    df[name] = df[name].apply(lambda x: " ".join(x for x in  str(x).split() if len(x) > 3))
    return df
def word_count(df, text, name):
    df[name] = df[text].apply(lambda x: len(str(x).split(" ")))
    return df

def hashtags_count(df, text, name):
    df[name] = df[text].apply(lambda x: len([x for x in  str(x).split() if x.startswith('#')]))
    return df

def user_mentions_count(df, text, name):
    df[name] = df[text].apply(lambda x: len([x for x in  str(x).split() if x.startswith('@')]))
    return df

def numerics_count(df, text, name):
    df[name] = df[text].apply(lambda x: len([x for x in  str(x).split() if x.isdigit()]))
    return df

def links_count(df, text, name):
    df[name] = df[text].apply(
        lambda x: len([x for x in  str(x).split() if x.startswith(('httP', 'HTTPS', 'https', 'www', 'WWW', 'HTTP', 'http')) ]))
    return df

# **Read From CSV File** 



In [6]:
df = pd.read_csv("drive/My Drive/Output.csv",encoding='latin1',low_memory=False)
df=df[:40000]

# clean review Content
clean(df, "reviewContent", "cleanContent")

,reviewID,reviewerID,reviewContent,rating,usefulCount,date,reviewerID.1,firstCount,reviewCount,Filtered,cleanContent
0,---2eJpQ2HAnRQRp2v7XLA,-ORdirlxcallT7dzlZlW2A,I came to Chicago on business and was initiall...,5,5,9/28/2011,-ORdirlxcallT7dzlZlW2A,0.0,2.0,0.0,came chicago business initially supposed stay ...
1,--0gvJ0LlByoUsFi6x4eEQ,biZ0fIcZp7B-KBJ3EGcX7Q,I still love this place but the took Mu Shu of...,5,2,Update - 5/12/2009,biZ0fIcZp7B-KBJ3EGcX7Q,268.0,543.0,0.0,still love place took menu deeply saddened sho...
2,--1PwTt0AfyPG1pE7bB6Nw,pW9LvtC-hZ0PRAOUn7aZ2A,Alan is the best CPA that I know. He knows all...,5,0,8/15/2006,pW9LvtC-hZ0PRAOUn7aZ2A,834.0,1463.0,0.0,alan best know knows outs loop holes smart whi...
3,--87QVdPvgbDK3GeVxFRzg,92rtlWxS2aOngzkKhi-OxQ,My family has been using Korean Air for many m...,4,0,7/28/2012,92rtlWxS2aOngzkKhi-OxQ,43.0,158.0,0.0,family using korean many many years going asia...
4,--99ZJoVHP32iSd4i4nlZw,Egu4qSFR56EhGuTHZTaVjA,I hadn't been to Grisanti's since a pre-homeco...,3,0,1/28/2010,Egu4qSFR56EhGuTHZTaVjA,133.0,725.0,0.0,hadnt grisantis since prehomecoming dinner hig...
...,...,...,...,...,...,...,...,...,...,...,...
39995,fNlm6JHAHcSe-QROn7O6VQ,drzHAHTHFnThHW0R0rOdDQ,***Si Senor is now called Wapos*** Food is pre...,4,1,9/9/2012,drzHAHTHFnThHW0R0rOdDQ,24.0,405.0,0.0,senor called wapos food pretty good prices rig...
39996,FNLp_jD_sY0R-yzc7Lc0GA,0G-QF457q_0Z_jKqh6xWiA,I've resisted trying Italian restaurants in Ka...,3,8,12/22/2010,0G-QF457q_0Z_jKqh6xWiA,95.0,1010.0,0.0,resisted trying italian restaurants kansas cit...
39997,FnlT8PuD86KVZ4f3o6Hemg,Bx6Tz_TJDS7KB5jX123haA,Have only dined in once for lunch thought the ...,3,0,2/11/2012,Bx6Tz_TJDS7KB5jX123haA,13.0,177.0,0.0,dined lunch thought food wasnt wild reviewers ...
39998,fnl_AyDHUGFPRhLFGIRy8A,0Igx-a1wAstiBDerGxXk2A,So say you're trying to do an event on a budge...,4,4,1/18/2011,0Igx-a1wAstiBDerGxXk2A,302.0,917.0,0.0,youre trying event budget want purchase bulk f...


In [80]:
df.info()
df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   reviewID           40000 non-null  object 
 1   reviewerID         40000 non-null  object 
 2   reviewContent      40000 non-null  object 
 3   rating             40000 non-null  object 
 4   usefulCount        40000 non-null  object 
 5   date               39994 non-null  object 
 6   reviewerID.1       39994 non-null  object 
 7   firstCount         39994 non-null  float64
 8   reviewCount        39994 non-null  float64
 9   Filtered           39994 non-null  float64
 10  cleanContent       40000 non-null  object 
 11  linksCount         40000 non-null  int64  
 12  hashtagsCount      40000 non-null  int64  
 13  userMentionsCount  40000 non-null  int64  
 14  numericsCount      40000 non-null  int64  
 15  wCount             40000 non-null  int64  
 16  wordCount          400

Index(['reviewID', 'reviewerID', 'reviewContent', 'rating', 'usefulCount',
       'date', 'reviewerID.1', 'firstCount', 'reviewCount', 'Filtered',
       'cleanContent', 'linksCount', 'hashtagsCount', 'userMentionsCount',
       'numericsCount', 'wCount', 'wordCount'],
      dtype='object')

# **TF-IDF**

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
tfidf_matrix = tfidfconverter.fit_transform(df["cleanContent"])

In [8]:
print(len(tfidf_matrix.toarray()))


40000


# **calculateing cosine similarity**

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
cosineSim=[]
for i in range(len(tfidf_matrix.toarray())):
  sim=(cosine_similarity(tfidf_matrix[i], tfidf_matrix)).mean()
  cosineSim.append(sim)

# **text processing**

In [10]:
list_of_rows = [list(row) for row in df.values]
#print(len(cosineSim))
links_count(df,"reviewContent","linksCount")
hashtags_count(df, "reviewContent", "hashtagsCount")
user_mentions_count(df, "reviewContent", "userMentionsCount")
numerics_count(df,"reviewContent","numericsCount")
clean(df, "reviewContent", "cleanContent")

word_count(df, "reviewContent", "wCount")
word_count(df, "cleanContent", "wordCount")



,reviewID,reviewerID,reviewContent,rating,usefulCount,date,reviewerID.1,firstCount,reviewCount,Filtered,cleanContent,linksCount,hashtagsCount,userMentionsCount,numericsCount,wCount,wordCount
0,---2eJpQ2HAnRQRp2v7XLA,-ORdirlxcallT7dzlZlW2A,I came to Chicago on business and was initiall...,5,5,9/28/2011,-ORdirlxcallT7dzlZlW2A,0.0,2.0,0.0,came chicago business initially supposed stay ...,0,0,0,0,138,70
1,--0gvJ0LlByoUsFi6x4eEQ,biZ0fIcZp7B-KBJ3EGcX7Q,I still love this place but the took Mu Shu of...,5,2,Update - 5/12/2009,biZ0fIcZp7B-KBJ3EGcX7Q,268.0,543.0,0.0,still love place took menu deeply saddened sho...,0,0,0,0,21,10
2,--1PwTt0AfyPG1pE7bB6Nw,pW9LvtC-hZ0PRAOUn7aZ2A,Alan is the best CPA that I know. He knows all...,5,0,8/15/2006,pW9LvtC-hZ0PRAOUn7aZ2A,834.0,1463.0,0.0,alan best know knows outs loop holes smart whi...,0,0,0,0,102,28
3,--87QVdPvgbDK3GeVxFRzg,92rtlWxS2aOngzkKhi-OxQ,My family has been using Korean Air for many m...,4,0,7/28/2012,92rtlWxS2aOngzkKhi-OxQ,43.0,158.0,0.0,family using korean many many years going asia...,0,0,0,0,135,62
4,--99ZJoVHP32iSd4i4nlZw,Egu4qSFR56EhGuTHZTaVjA,I hadn't been to Grisanti's since a pre-homeco...,3,0,1/28/2010,Egu4qSFR56EhGuTHZTaVjA,133.0,725.0,0.0,hadnt grisantis since prehomecoming dinner hig...,0,0,0,0,142,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,fNlm6JHAHcSe-QROn7O6VQ,drzHAHTHFnThHW0R0rOdDQ,***Si Senor is now called Wapos*** Food is pre...,4,1,9/9/2012,drzHAHTHFnThHW0R0rOdDQ,24.0,405.0,0.0,senor called wapos food pretty good prices rig...,0,0,0,0,33,16
39996,FNLp_jD_sY0R-yzc7Lc0GA,0G-QF457q_0Z_jKqh6xWiA,I've resisted trying Italian restaurants in Ka...,3,8,12/22/2010,0G-QF457q_0Z_jKqh6xWiA,95.0,1010.0,0.0,resisted trying italian restaurants kansas cit...,0,0,0,0,143,69
39997,FnlT8PuD86KVZ4f3o6Hemg,Bx6Tz_TJDS7KB5jX123haA,Have only dined in once for lunch thought the ...,3,0,2/11/2012,Bx6Tz_TJDS7KB5jX123haA,13.0,177.0,0.0,dined lunch thought food wasnt wild reviewers ...,0,0,0,0,24,9
39998,fnl_AyDHUGFPRhLFGIRy8A,0Igx-a1wAstiBDerGxXk2A,So say you're trying to do an event on a budge...,4,4,1/18/2011,0Igx-a1wAstiBDerGxXk2A,302.0,917.0,0.0,youre trying event budget want purchase bulk f...,0,0,0,0,78,32


# **feature Extraction**

In [11]:
dateList=[]
for word in list_of_rows:
    if "Update" in str(word[5]):
        dateList.append(1)
    else:
        dateList.append(0)

extremeRating=[]
for rating in list(df["rating"]):
    if rating == 5 or rating == 1:
        extremeRating.append(1)
    else:
        extremeRating.append(0)




dict = {'UpdateOrNot': dateList,
       'NumberOfURLs': list(df["linksCount"]),
       'NumberOfnumerics': list(df["numericsCount"]),
      #'hashtagsCount': list(df["hashtagsCount"]),
      #'userMentionsCount': list(df["userMentionsCount"]),
       'extremeRating':extremeRating,
       'firstCount':list(df["firstCount"]),
       'wordCount':list(df['wCount']),
       'reviewCount':list(df['reviewCount']),
       'usefulCount':list(df['usefulCount']),
        'cosineSimilarty':cosineSim,
       'Filtered':list(df["Filtered"])
        }


df1 = pd.DataFrame(dict)
df1.to_csv('Data.csv',index=False)
df1.head()

,UpdateOrNot,NumberOfURLs,NumberOfnumerics,extremeRating,firstCount,wordCount,reviewCount,usefulCount,cosineSimilarty,Filtered
0,0,0,0,0,0.0,138,2.0,5,0.038284,0.0
1,1,0,0,0,268.0,21,543.0,2,0.027823,0.0
2,0,0,0,0,834.0,102,1463.0,0,0.021433,0.0
3,0,0,0,0,43.0,135,158.0,0,0.043143,0.0
4,0,0,0,0,133.0,142,725.0,0,0.043215,0.0


In [12]:
df1.info()
df1['Filtered'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   UpdateOrNot       40000 non-null  int64  
 1   NumberOfURLs      40000 non-null  int64  
 2   NumberOfnumerics  40000 non-null  int64  
 3   extremeRating     40000 non-null  int64  
 4   firstCount        39994 non-null  float64
 5   wordCount         40000 non-null  int64  
 6   reviewCount       39994 non-null  float64
 7   usefulCount       40000 non-null  object 
 8   cosineSimilarty   40000 non-null  float64
 9   Filtered          39994 non-null  float64
dtypes: float64(4), int64(5), object(1)
memory usage: 3.1+ MB


0.0    33220
1.0     6774
Name: Filtered, dtype: int64

In [13]:
df1.replace([np.inf, -np.inf], np.nan, inplace=True)
df1= df1.apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna().astype(int)

In [14]:
X = df1.drop('Filtered', axis=1).astype(int, errors='ignore')
y = df1['Filtered'].astype(int, errors='ignore')

# **handling imbalanced dataset by over sampling**

In [22]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

oversample = RandomOverSampler(sampling_strategy='minority')
oversample = RandomOverSampler(sampling_strategy=1.0)
X, y = oversample.fit_resample(X, y)
print(Counter(y))



Counter({0: 33220, 1: 33220})


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


# **Split for train and test**

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y ,test_size=0.3)


# **Decision Tree Classifier**

In [17]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print("Decision Tree:")
print("Accuracy:",accuracy_score(y_test, y_pred)*100)
print("precision_score:",precision_score(y_test, y_pred)*100)
print("recall_score:",recall_score(y_test, y_pred)*100)
print("f1_score:",f1_score(y_test, y_pred)*100)


Decision Tree:
Accuracy: 89.1782059000602
precision_score: 83.78308251167617
recall_score: 97.17123081552815
f1_score: 89.98188658213738


# **xgboost Classifier**

In [18]:
import xgboost
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

model = xgboost.XGBClassifier(subsample=0.85,learning_rate=0.3,max_depth= 95,max_features=1,min_samples_leaf=3,min_samples_split= 3,n_estimators= 1000)
kfold = StratifiedKFold(n_splits=5, random_state=None, shuffle=True)
for score in ["accuracy", "precision", "recall","f1"]:
  print( score," : ",cross_val_score(model, X,y,scoring=score, cv=kfold).mean()*100)

accuracy  :  92.93046357615896
precision  :  88.7418297450717
recall  :  98.39253461770019
f1  :  93.42213598087964


# **export**



In [75]:
import pickle


In [76]:

with open('drive/My Drive/model_pickle','wb') as file:
    pickle.dump(classifier,file)

In [33]:
with open('model_pickle','rb') as file:
    mp = pickle.load(file)